# Observation et premières analyses des données Network

In [1]:
# Librairies
import pandas as pd

from sklearn.preprocessing import LabelEncoder

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.subplots as sp
from plotly.graph_objs import Bar
import plotly.express as px

## Chargement des données

Nous chargeons les 5 fichiers de données de réseau csv.

In [ ]:
df_net_1 = pd.read_csv('datasets/Network datatset/csv/attack_1.csv')
df_net_2 = pd.read_csv('datasets/Network datatset/csv/attack_2.csv')
df_net_3 = pd.read_csv('datasets/Network datatset/csv/attack_3.csv')
df_net_4 = pd.read_csv('datasets/Network datatset/csv/attack_4.csv')
df_net_norm = pd.read_csv('datasets/Network datatset/csv/normal.csv')

Nous commençons par observer le premier, comme ils doivent à priori avoir la même structure, pour comprendre les données que l'on a.

## Observation des données du premier fichier

In [3]:
df_net_1.head()

,Time,mac_s,mac_d,ip_s,ip_d,sport,dport,proto,flags,size,modbus_fn,n_pkt_src,n_pkt_dst,modbus_response,label_n,label
0,2021-04-09 18:23:28.385003,74:46:a0:bd:a7:1b,0a:fe:ec:47:74:fb,84.3.251.20,84.3.251.102,56667.0,502.0,Modbus,11000.0,66,Read Coils Request,0.0,0.0,NaN,0,normal
1,2021-04-09 18:23:28.385005,74:46:a0:bd:a7:1b,e6:3f:ac:c9:a8:8c,84.3.251.20,84.3.251.101,56666.0,502.0,Modbus,11000.0,66,Read Coils Request,1.0,0.0,NaN,0,normal
2,2021-04-09 18:23:28.385006,74:46:a0:bd:a7:1b,fa:00:bc:90:d7:fa,84.3.251.20,84.3.251.103,56668.0,502.0,Modbus,11000.0,66,Read Coils Request,2.0,0.0,NaN,0,normal
3,2021-04-09 18:23:28.385484,0a:fe:ec:47:74:fb,74:46:a0:bd:a7:1b,84.3.251.102,84.3.251.20,502.0,56667.0,Modbus,11000.0,64,Read Coils Response,0.0,0.0,[0],0,normal
4,2021-04-09 18:23:28.385486,fa:00:bc:90:d7:fa,74:46:a0:bd:a7:1b,84.3.251.103,84.3.251.20,502.0,56668.0,Modbus,11000.0,64,Read Coils Response,0.0,1.0,[0],0,normal


In [4]:
df_net_1.shape

(5527409, 16)

In [5]:
df_net_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5527409 entries, 0 to 5527408
Data columns (total 16 columns):
 #   Column            Dtype  
---  ------            -----  
 0   Time              object 
 1    mac_s            object 
 2    mac_d            object 
 3    ip_s             object 
 4    ip_d             object 
 5    sport            float64
 6    dport            float64
 7    proto            object 
 8    flags            float64
 9    size             int64  
 10   modbus_fn        object 
 11   n_pkt_src        float64
 12   n_pkt_dst        float64
 13   modbus_response  object 
 14   label_n          int64  
 15   label            object 
dtypes: float64(5), int64(2), object(9)
memory usage: 674.7+ MB


Le dataset réseau contient des informations détaillées sur le trafic Modbus :
- adresses MAC et IP source et destination
- les ports utilisés
- le protocole de communication
- des flags TCP
- la taille des paquets 
- codes fonctionnels Modbus
- le nombre de paquets provenant de la même source ou destination
- la réponse du protocole Modbub
- le label indiquant si le paquet est normal ou malveillant

### Analyse des différentes colonnes

In [6]:
# Colonnes label et label_n
df_net_1[' label'].value_counts()

 label
normal            3687410
MITM              1214098
physical fault     625691
anomaly               210
Name: count, dtype: int64

- normal : pas d'attaque
- MITM (= Man-in-the-Middle) : L'attaquant intercepte et peut modifier les communications entre deux appareils.
- physical fault : Anomalies physiques non causée par une cyberattaque \
--> Normalement non détectables dans les données réseau
- anomaly : autre anomalies non MITM et pas un défaut physique.

In [7]:
df_net_1[' label_n'].value_counts()

 label_n
0    3687410
1    1839999
Name: count, dtype: int64

In [8]:
print(df_net_1[[' label', ' label_n']].value_counts())

 label           label_n
normal          0           3687410
MITM            1           1214098
physical fault  1            625691
anomaly         1               210
Name: count, dtype: int64


normal : label_n = 0 \
attack : label_n = 1

In [9]:
# Colonne Time
df_time = pd.to_datetime(df_net_1['Time'], format="mixed") # mixed : car format de la date pas toujours le même
df_time.dtype

dtype('<M8[ns]')

In [10]:
# Nombre de valeurs enregistrées pour un temps donné
df_time.value_counts()

Time
2021-04-09 18:42:37.177930    4
2021-04-09 18:42:42.262608    4
2021-04-09 18:42:41.847931    4
2021-04-09 18:42:37.728146    4
2021-04-09 18:42:30.305264    4
                             ..
2021-04-09 18:37:08.030770    1
2021-04-09 18:37:08.030134    1
2021-04-09 18:37:08.029902    1
2021-04-09 18:37:08.029055    1
2021-04-09 19:03:47.661291    1
Name: count, Length: 5242099, dtype: int64

In [11]:
# Temps entre chaque ligne (chaque enregistrement)
df_time.diff().value_counts()

Time
0 days 00:00:00.000001    996481
0 days 00:00:00.000003    641614
0 days 00:00:00.000002    608511
0 days 00:00:00           285310
0 days 00:00:00.000015     73069
                           ...  
0 days 00:00:00.003579         1
0 days 00:00:00.003685         1
0 days 00:00:00.003622         1
0 days 00:00:00.002989         1
0 days 00:00:00.011012         1
Name: count, Length: 3399, dtype: int64

L'acquisition des données n'est pas à la même fréquence que celle des données physiques. En effet, les données physiques sont enregistrées toutes les 1s, tandis qu'ici, il n'y a pas d'enregistrement continue. 

Les données sont enregistrées lorsqu'il y a une interraction réseau.

## Analyse des différents types de colonnes

In [12]:
# Colonnes de chaque type
df_obj = df_net_1.select_dtypes(include='object')
print("Colonnes de type objet : \n",df_obj.columns,"\n")

df_num = df_net_1.select_dtypes(include='number')
print("Colonnes de type nombre : \n", df_num.columns)  

Colonnes de type objet : 
 Index(['Time', ' mac_s', ' mac_d', ' ip_s', ' ip_d', ' proto', ' modbus_fn',
       ' modbus_response', ' label'],
      dtype='object') 

Colonnes de type nombre : 
 Index([' sport', ' dport', ' flags', ' size', ' n_pkt_src', ' n_pkt_dst',
       ' label_n'],
      dtype='object')


### Analyse des colonnes numériques

In [13]:
df_num.describe()

,sport,dport,flags,size,n_pkt_src,n_pkt_dst,label_n
count,5.526894e+06,5.526894e+06,5.526894e+06,5.527409e+06,5.526934e+06,5.526934e+06,5.527409e+06
mean,2.849674e+04,2.837012e+04,1.091689e+04,6.532781e+01,3.070677e+01,3.074328e+01,3.328863e-01
std,2.796624e+04,2.798476e+04,8.398618e+02,1.400719e+00,1.756813e+01,1.753642e+01,4.712463e-01
min,2.500000e+01,2.500000e+01,1.000000e+01,6.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00
25%,5.020000e+02,5.020000e+02,1.100000e+04,6.500000e+01,1.500000e+01,1.500000e+01,0.000000e+00
50%,3.499100e+04,5.020000e+02,1.100000e+04,6.600000e+01,1.900000e+01,1.900000e+01,0.000000e+00
75%,5.666700e+04,5.666700e+04,1.100000e+04,6.600000e+01,5.000000e+01,5.000000e+01,1.000000e+00
max,6.099900e+04,6.099900e+04,1.100000e+04,7.800000e+01,5.300000e+01,5.200000e+01,1.000000e+00


In [14]:
# Suppression de la colonne count pour la visualisation
desc_stats = df_num.describe()
desc_stats = desc_stats.drop('count')

# Création d'un sous graphe pour chaque colonne
fig = make_subplots(rows=1, cols=len(desc_stats.columns), subplot_titles=desc_stats.columns)
for i, col in enumerate(desc_stats.columns):
    fig.add_trace(go.Bar(x=desc_stats.index, y=desc_stats[col], name=col),row=1, col=i+1)

fig.update_layout(height=700, width=1200,title="Statistiques descriptives par colonne",showlegend=False)

fig.show()

In [15]:
for col in df_num.columns:
    print(col, df_num[col].value_counts())

 sport  sport
502.0      2755878
56667.0     821093
56666.0     807888
56668.0     807710
56665.0     241775
            ...   
48341.0          3
47947.0          3
48843.0          3
53611.0          3
25.0             2
Name: count, Length: 8445, dtype: int64
 dport  dport
502.0      2771010
56667.0     821092
56666.0     807887
56668.0     807708
56665.0     241785
            ...   
34425.0          5
59477.0          5
46049.0          5
25.0             4
1027.0           2
Name: count, Length: 8441, dtype: int64
 flags  flags
11000.0    5383646
10000.0      53601
10.0         26410
10010.0      26409
10001.0      26361
100.0         5361
10100.0       5106
Name: count, dtype: int64
 size  size
66    2733117
65    1426127
64    1257165
60      51806
74      42633
78      12614
77       3947
Name: count, dtype: int64
 n_pkt_src  n_pkt_src
15.0    1515089
50.0    1074699
49.0     453662
51.0     434475
14.0     377759
13.0     202141
4.0      197631
44.0     139127
45.0     128248

In [16]:
columns = df_num.columns
titles = ["Sport", "Dport", "Flags", "Size", "n_pkt_src", "n_pkt_dst", "Label_n"]

# Création de sous graphique pour chaque colonne
fig = sp.make_subplots(rows=4, cols=2, subplot_titles=titles, vertical_spacing=0.1, horizontal_spacing=0.15)
for i, col in enumerate(columns):
    row = (i // 2) + 1
    col_position = (i % 2) + 1
    fig.add_trace(Bar(x=df_num[col].value_counts().index, y=df_num[col].value_counts().values, name=titles[i]),row=row, col=col_position)

fig.update_layout(height=700, width=700, title_text="Distribution des valeurs dans les colonnes numériques",showlegend=False)

fig.show()

- sport
    - Valeur moyenne ~30000, médiane un peu plus faible
    - Distribution : Forte concentration de valeurs spécifiques (port 502 par exemple qui est spécifique aux Modbus), certains ports moins représentés pourrait être des accès interdits


- dport
    - Comme pour sport, moyenne et un maximum élevés, et une médiane plus basse
    - Comme pour sport, la majorité des paquets sont concentrés sur quelques ports de destination, avec le port 502 comme destination principale. Des valeurs plus rares sont observées, mais elles sont peu fréquentes

- flags
    - La plupart des valeurs entre 8000 et 11000, avec une forte concentration autour de valeurs spécifiques (états standards de connexion ??)
    - La majorité des paquets ont des flags identiques, ce qui est attendu pour des communications industrielles stables. Quelques variations existent, ce qui peut représenter des tentatives d’intrusion ou des anomalies.

- size
    - Tailles de paquets concentrées autour de valeurs entre 64 et 66 octets + peu de variation
    - Distribution fortement centrée sur 66 octets (taille de paquets standard pour les échanges de données Modbus ??) \
    --> paquets d'autres tailles pourraient être intéressants à analyser pour des comportements anormaux

- n_pkt_src
    - Le nombre moyen de paquets par source est autour de 30-40, avec quelques certains allant jusqu'à 50
    - La distribution présente des pics élevés pour certaines valeurs spécifiques (flux réguliers de données ?? surcharge ou un potentiel DoS ???)

- n_pkt_dst
    - Comme pour n_pkt_src autour de 30-40, avec des valeurs maximales proches de 50
    - Pics dans la distribution indiquent des destinations qui reçoivent un grand nombre de paquets ?? ou anomalies ??

- label_n
    - Valeurs sont binaires (0 ou 1), avec 0 = paquets normaux et 1 attaque
    -La majorité des paquets sont normaux, donc dataset déséquilibré MAIS représentatif ?

### Analyse des colonnes de type objet

In [17]:
# Afficher les valeurs uniques pour chaque colonne
df_obj_no_time = df_obj.drop('Time', axis=1)
for col in df_obj_no_time.columns:
    print(col, df_obj_no_time[col].value_counts())

 mac_s  mac_s
74:46:a0:bd:a7:1b    2678457
0a:fe:ec:47:74:fb     853232
e6:3f:ac:c9:a8:8c     850385
fa:00:bc:90:d7:fa     833496
00:80:f4:03:fb:12     276589
fe:bb:16:7b:c3:27      14801
4a:35:83:e0:3d:a4      14661
00:0c:29:47:8c:22       5788
Name: count, dtype: int64
 mac_d  mac_d
74:46:a0:bd:a7:1b    2678456
0a:fe:ec:47:74:fb     858271
e6:3f:ac:c9:a8:8c     853239
fa:00:bc:90:d7:fa     833390
00:80:f4:03:fb:12     276586
fe:bb:16:7b:c3:27      10942
4a:35:83:e0:3d:a4      10784
00:0c:29:47:8c:22       5597
ff:ff:ff:ff:ff:ff        144
Name: count, dtype: int64
 ip_s  ip_s
84.3.251.20     2678448
84.3.251.102     854860
84.3.251.101     851692
84.3.251.103     834158
84.3.251.18      277499
84.3.251.104      15486
84.3.251.105      14791
Name: count, dtype: int64
 ip_d  ip_d
84.3.251.20     2678456
84.3.251.102     860155
84.3.251.101     854680
84.3.251.103     833855
84.3.251.18      277494
84.3.251.104      11352
84.3.251.105      10942
Name: count, dtype: int64
 proto  proto
M

In [18]:
df_obj_rep = df_obj.drop('Time', axis=1) # Pas de sens pour cette colonne

fig = make_subplots(rows=1, cols=len(df_obj_rep.columns), subplot_titles=df_obj_rep.columns)

# Sous graphique pour chaque colonne
for i, col in enumerate(df_obj_rep.columns):
    value_counts = df_obj_rep[col].value_counts().head(10)  # Pour éviter d'avoir trop de valeurs
    fig.add_trace(go.Bar(x=value_counts.index, y=value_counts.values, name=col), row=1, col=i+1)


fig.update_layout(height=700, width=1300, title="Fréquence des valeurs uniques par colonne (object)", showlegend=False)

fig.show()

- mac_s, mac_d : 
    - une adresse principale largement dominante 
    - trafic centré autour de quelques appareils
    - autres sont peut-être anormales

- ip_s, ip_d : 
    - adresses IP source et destination également dominées par quelques valeurs
    - communication entre des hôtes spécifiques

- proto : 
    - protocole Modbus de loin le plus utilisé,
    - confirme l’utilisation majoritaire de Modbus dans le système

- modbus_fn : 
    - rien de spécial, ce sont les fonctions Modbus, toutes utilisées de façon plus ou moins équivalentes

- modbus_response : 
    - réponse [0] dominante, surement représentant une exécution correcte
    - autres valeurs beaucoup moins présentes

- label : 
    - majorité des données marquées comme normales
    - autres valeurs non normales

## Comparaison des fichiers de données

In [19]:
# Taille des fichiers
print("Taille du fichier 1 : ", df_net_1.shape)
print("Taille du fichier 2 : ", df_net_2.shape)
print("Taille du fichier 3 : ", df_net_3.shape)
print("Taille du fichier 4 : ", df_net_4.shape)
print("Taille du fichier normal : ", df_net_norm.shape)

Taille du fichier 1 :  (5527409, 16)
Taille du fichier 2 :  (5159469, 16)
Taille du fichier 3 :  (5862547, 16)
Taille du fichier 4 :  (5522490, 16)
Taille du fichier normal :  (7757289, 16)


Les fichiers d'attaques font en moyenne la même taille, et le fichier normal est plus grand.

In [20]:
# Comparaison des noms de colonnes
dataframes = [df_net_1, df_net_2, df_net_3, df_net_4, df_net_norm]

column_names_by_index = {}

for i, df in enumerate(dataframes):
    for col_index, col_name in enumerate(df.columns):
        if col_index not in column_names_by_index:
            column_names_by_index[col_index] = []
        column_names_by_index[col_index].append(col_name)

for index, col_names in column_names_by_index.items():
    col_count = pd.Series(col_names).value_counts()
    print(f"Colonne n°{index}:")
    for col_name, count in col_count.items():
        print(f"\"{col_name}\"  {count}")
    print()

Colonne n°0:
"Time"  5

Colonne n°1:
" mac_s"  3
"mac_s"  2

Colonne n°2:
" mac_d"  3
"mac_d"  2

Colonne n°3:
" ip_s"  3
"ip_s"  2

Colonne n°4:
" ip_d"  3
"ip_d"  2

Colonne n°5:
" sport"  3
"sport"  2

Colonne n°6:
" dport"  3
"dport"  2

Colonne n°7:
" proto"  3
"proto"  2

Colonne n°8:
" flags"  3
"flags"  2

Colonne n°9:
" size"  3
"size"  2

Colonne n°10:
" modbus_fn"  3
"modbus_fn"  2

Colonne n°11:
" n_pkt_src"  3
"modbus_response"  1
"n_pkt_src"  1

Colonne n°12:
" n_pkt_dst"  3
"n_pkt_src"  1
"n_pkt_dst"  1

Colonne n°13:
" modbus_response"  3
"n_pkt_dst"  1
"modbus_response"  1

Colonne n°14:
" label_n"  3
"label_n"  2

Colonne n°15:
" label"  3
"label"  2



On peut voir que 3 sur 5 des noms de colonnes ont un espace au début, il faudra le supprimer par la suite. De plus, 1 des fichiers a deux colonnes inversées.

In [21]:
print(df_net_1[' label_n'].value_counts(), "\n")
print(df_net_2[' label_n'].value_counts(), "\n")
print(df_net_3[' label_n'].value_counts(),  "\n")
print(df_net_4['label_n'].value_counts(), "\n")
print(df_net_norm['label_n'].value_counts(), "\n")

 label_n
0    3687410
1    1839999
Name: count, dtype: int64 

 label_n
0    4093168
1    1066301
Name: count, dtype: int64 

 label_n
1    3791992
0    2070555
Name: count, dtype: int64 

label_n
0    2844877
1    2677613
Name: count, dtype: int64 

label_n
0    7757289
Name: count, dtype: int64 



In [22]:
print(df_net_1[[' label', ' label_n']].value_counts(), "\n")
print(df_net_2[[' label', ' label_n']].value_counts(), "\n")
print(df_net_3[[' label', ' label_n']].value_counts(),  "\n")
print(df_net_4[['label', 'label_n']].value_counts(), "\n")
print(df_net_norm[['label', 'label_n']].value_counts(), "\n")

 label           label_n
normal          0           3687410
MITM            1           1214098
physical fault  1            625691
anomaly         1               210
Name: count, dtype: int64 

 label           label_n
normal          0           4093168
DoS             1            571875
physical fault  1            277282
MITM            1            217009
anomaly         1               105
scan            1                30
Name: count, dtype: int64 

 label           label_n
DoS             1           3194711
normal          0           2070555
physical fault  1            344244
MITM            1            252963
anomaly         1                74
Name: count, dtype: int64 

label           label_n
normal          0          2844877
DoS             1          1904956
MITM            1           471339
physical fault  1           301287
scan            1               31
Name: count, dtype: int64 

label   label_n
normal  0          7757289
Name: count, dtype: int64 



Le fichier normal.csv ne contient pas de label = 1, contrairement aux fichiers d'attaque. \
Le fichier est donc bien un fichier représentant le fonctionnement normale des échanges réseau.

On peut retrouver d'autres types d'attaques dans les 3 fichiers non analysés avant.
- DoS (=Denial of service) : L'attaquant inonde le service de requêtes/paquets pour le surcharger et le rendre indisponible.
- scan : L'attaquant cherche à identifier les ports ouverts et les vulnérabilités potentielles avant une attaque.

## Nettoyage des données

### Noms colonnes

In [23]:
# Correction des noms de colonnes (suppression des espaces)
dataframes = [df_net_1, df_net_2, df_net_3, df_net_4, df_net_norm]
for df in dataframes:
    df.columns = df.columns.str.strip()

In [24]:
# Vérification des noms de colonnes
dataframes = [df_net_1, df_net_2, df_net_3, df_net_4, df_net_norm]

column_names_by_index = {}

for i, df in enumerate(dataframes):
    for col_index, col_name in enumerate(df.columns):
        if col_index not in column_names_by_index:
            column_names_by_index[col_index] = []
        column_names_by_index[col_index].append(col_name)

for index, col_names in column_names_by_index.items():
    col_count = pd.Series(col_names).value_counts()
    print(f"Colonne n°{index}:")
    for col_name, count in col_count.items():
        print(f"\"{col_name}\"  {count}")
    print()

Colonne n°0:
"Time"  5

Colonne n°1:
"mac_s"  5

Colonne n°2:
"mac_d"  5

Colonne n°3:
"ip_s"  5

Colonne n°4:
"ip_d"  5

Colonne n°5:
"sport"  5

Colonne n°6:
"dport"  5

Colonne n°7:
"proto"  5

Colonne n°8:
"flags"  5

Colonne n°9:
"size"  5

Colonne n°10:
"modbus_fn"  5

Colonne n°11:
"n_pkt_src"  4
"modbus_response"  1

Colonne n°12:
"n_pkt_dst"  4
"n_pkt_src"  1

Colonne n°13:
"modbus_response"  4
"n_pkt_dst"  1

Colonne n°14:
"label_n"  5

Colonne n°15:
"label"  5



### Format colonne Time

In [25]:
# Mise en forme uniforme de la colonne Time
dataframes = [df_net_1, df_net_2, df_net_3, df_net_4, df_net_norm]

for df in dataframes:
    df['Time'] = pd.to_datetime(df['Time'], errors='coerce')
    df['Time'] = df['Time'].dt.strftime('%Y-%m-%d %H:%M:%S.%f')

### Gestion des valeurs manquantes

#### Recherche

In [26]:
# Affichage des valeurs null dans chaque colonnes de chaque df
dataframes = [df_net_1, df_net_2, df_net_3, df_net_4, df_net_norm]

for i, df in enumerate(dataframes, 1):
    print(f"Valeurs nulles dans df_net_{i} :")
    null_counts = df.isnull().sum()
    print(null_counts)
    print()

Valeurs nulles dans df_net_1 :
Time                     3
mac_s                    0
mac_d                    0
ip_s                   475
ip_d                   475
sport                  515
dport                  515
proto                    0
flags                  515
size                     0
modbus_fn           153123
n_pkt_src              475
n_pkt_dst              475
modbus_response    2840182
label_n                  0
label                    0
dtype: int64

Valeurs nulles dans df_net_2 :
Time                     3
mac_s                    0
mac_d                    0
ip_s                   276
ip_d                   276
sport               385383
dport               385383
proto                    0
flags               385383
size                     0
modbus_fn           539517
n_pkt_src              276
n_pkt_dst              276
modbus_response    2849430
label_n                  0
label                    0
dtype: int64

Valeurs nulles dans df_net_3 :
Time           

#### Remplacement / Suppression (idées à revoir)

In [27]:
dataframes = [df_net_1, df_net_2, df_net_3, df_net_4, df_net_norm]

for df in dataframes:
    # Suppression des lignes avec Time nul car essentiel pour l'analyse
    df['Time'] = df['Time'].ffill()

    # Remplacement des ip null par "Inconnue"
    df['ip_s'] = df['ip_s'].fillna('Inconnue')
    df['ip_d'] = df['ip_d'].fillna('Inconnue')

    # Remplacement des port null par -1
    df['sport'] = df['sport'].fillna(-1)
    df['dport'] = df['dport'].fillna(-1)

    # Remplacement des flags null par -1
    df['flags'] = df['flags'].fillna(-1)

    # Remplacement des échanges Modbus par "Inconnu"
    df['modbus_fn'] = df['modbus_fn'].fillna('Inconnu')

    # Remplacement du nombre de paquets par source/destination null par -1
    df['n_pkt_src'] = df['n_pkt_src'].fillna(-1)
    df['n_pkt_dst'] = df['n_pkt_dst'].fillna(-1)

    # Remplacement des réponses de Modbus null par "Pas de réponse"
    df['modbus_response'] = df['modbus_response'].fillna('Pas de réponse')


In [28]:
# Vérification
dataframes = [df_net_1, df_net_2, df_net_3, df_net_4, df_net_norm]

for i, df in enumerate(dataframes, 1):
    print(f"Valeurs nulles dans df_net_{i} :")
    null_counts = df.isnull().sum()
    print(null_counts[null_counts > 0]) # Affiche seulement les colonnes avec des valeurs nulles
    print()

Valeurs nulles dans df_net_1 :
Series([], dtype: int64)

Valeurs nulles dans df_net_2 :
Series([], dtype: int64)

Valeurs nulles dans df_net_3 :
Series([], dtype: int64)

Valeurs nulles dans df_net_4 :
Series([], dtype: int64)

Valeurs nulles dans df_net_5 :
Series([], dtype: int64)



## Matrice de corrélation car c'est jolie

### Matrice dans le dataset normal

In [29]:
df = df_net_norm

object_cols = df.select_dtypes(include=['object']).columns
le = LabelEncoder()
for col in object_cols:
    df[col] = le.fit_transform(df[col].astype(str))

correlation_matrix = df.corr()

#print(f"Matrice de corrélation pour le DataFrame {dataframes.index(df) + 1}")
#print(correlation_matrix)

fig = px.imshow(correlation_matrix, title="Matrice de corrélation du dataset normal", color_continuous_scale="blues", zmin=-1, zmax=1, height=600, width=600)
fig.show()

Nous pouvons voir que ip_s, sport et n_pkt_src sont très corrélés, de même pour ip_d, dport et n_pkt_dst. \
Cela semble plutot logique, car les échanges sont les mêmes.

In [30]:
df = df_net_1

object_cols = df.select_dtypes(include=['object']).columns
le = LabelEncoder()
for col in object_cols:
    df[col] = le.fit_transform(df[col].astype(str))

correlation_matrix = df.corr()

#print(f"Matrice de corrélation pour le DataFrame {dataframes.index(df) + 1}")
#print(correlation_matrix)

fig = px.imshow(correlation_matrix, title="Matrice de corrélation du dataset attack 1", color_continuous_scale="blues", zmin=-1, zmax=1, height=600, width=600)
fig.show()

Peu de différence avec le dataset normale, il va être compliqué de détecter des anomalies dans ce dataset.
Surement du au fait que les attaques sont physique en grande partie dans ce dataset, et donc non détectable sur des données réseaux.

In [31]:
df = df_net_2

object_cols = df.select_dtypes(include=['object']).columns
le = LabelEncoder()
for col in object_cols:
    df[col] = le.fit_transform(df[col].astype(str))

correlation_matrix = df.corr()

#print(f"Matrice de corrélation pour le DataFrame {dataframes.index(df) + 1}")
#print(correlation_matrix)

fig = px.imshow(correlation_matrix, title="Matrice de corrélation du dataset attack 2", color_continuous_scale="blues", zmin=-1, zmax=1, height=600, width=600)
fig.show()

In [32]:
df = df_net_3

object_cols = df.select_dtypes(include=['object']).columns
le = LabelEncoder()
for col in object_cols:
    df[col] = le.fit_transform(df[col].astype(str))

correlation_matrix = df.corr()

#print(f"Matrice de corrélation pour le DataFrame {dataframes.index(df) + 1}")
#print(correlation_matrix)

fig = px.imshow(correlation_matrix, title="Matrice de corrélation du dataset attack 3", color_continuous_scale="blues", zmin=-1, zmax=1, height=600, width=600)
fig.show()

In [33]:
df = df_net_4

object_cols = df.select_dtypes(include=['object']).columns
le = LabelEncoder()
for col in object_cols:
    df[col] = le.fit_transform(df[col].astype(str))

correlation_matrix = df.corr()

#print(f"Matrice de corrélation pour le DataFrame {dataframes.index(df) + 1}")
#print(correlation_matrix)

fig = px.imshow(correlation_matrix, title="Matrice de corrélation du dataset attack 4", color_continuous_scale="blues", zmin=-1, zmax=1, height=600, width=600)
fig.show()

Ces 3 dataset ont des matrices de corrélations assez différentes du dataset normale. \
Ces dataset ont des attaques physiques en moindre quantités comme vu précédemment, il est donc plus facile de détecter les attaques qui sont visibles sur les données réseau.